### MultiIntBond

In [1]:
%load_ext autoreload
%autoreload 2

In [43]:
from absbox import API,mkDeal,readInspect,EnginePath
localAPI = API(EnginePath.DEV,check=False)

Connecting engine server -> http://localhost:8081

✅Connected, local lib:0.28.5, server:0.40.13

In [159]:
from absbox import Generic

test01 = Generic(
    "MultiIntBond"
    ,{"cutoff":"2021-03-01","closing":"2021-06-15","firstPay":"2021-07-26"
     ,"payFreq":["DayOfMonth",20],"poolFreq":"MonthEnd","stated":"2030-01-01"}
    ,{'assets':[["Mortgage"
        ,{"originBalance":2200,"originRate":["fix",0.045],"originTerm":30
          ,"freq":"Monthly","type":"Level","originDate":"2021-02-01"}
          ,{"currentBalance":2200
          ,"currentRate":0.08
          ,"remainTerm":20
          ,"status":"current"}]]}
    ,(("acc01",{"balance":0}),)
    ,(("A1",{"balance":1200
             ,"rates":[0.05,0.02]
             ,"originBalance":1200
             ,"originRate":0.07
             ,"startDate":"2020-01-03"
             ,"rateTypes":[("fix",0.05),("fix",0.02)]
             ,"bondType":{"Sequential":None}}
       ) 
      ,("B",{"balance":1000
             ,"rate":0.0
             ,"originBalance":1000
             ,"originRate":0.07
             ,"startDate":"2020-01-03"
             ,"rateType":{"Fixed":0.00}
             ,"bondType":{"Equity":None}
             }))
    ,(("trusteeFee",{"type":{"fixFee":30}}),)
    ,{"amortizing":[
         ["payFee","acc01",['trusteeFee']]
         ,["calcInt","A1"]
         , ["inspect","after acc a1 all ",("bondDueIntTotal","A1")]
         , ["inspect","after acc a1 0 ",("bondDueIntByIndex",0,"A1")]
         , ["inspect","after acc a1 0 ioi",("bondDueIntOverIntByIndex",0,"A1")]
         , ["inspect","after acc a1 1 ",("bondDueIntByIndex",1,"A1")]
         , ["inspect","after acc a1 1 ioi",("bondDueIntOverIntByIndex",1,"A1")]
         ,["payIntByIndex","acc01",["A1"],0]
         , ["inspect","after pay 0 a1 all ",("bondDueIntTotal","A1")]
         , ["inspect","after pay 0 a1 0 ",("bondDueIntByIndex",0,"A1")]
         , ["inspect","after pay 0 a1 0 ioi",("bondDueIntOverIntByIndex",0,"A1")]
         , ["inspect","after pay 0 a1 1 ",("bondDueIntByIndex",1,"A1")]
         , ["inspect","after pay 0 a1 1 ioi",("bondDueIntOverIntByIndex",1,"A1")]
         ,["payIntByIndex","acc01",["A1"],1]
         , ["inspect","after pay 1 a1 all ",("bondDueIntTotal","A1")]
         , ["inspect","after pay 1 a1 0 ",("bondDueIntByIndex",0,"A1")]
         , ["inspect","after pay 1 a1 1 ",("bondDueIntByIndex",1,"A1")]
         #,["payPrin","acc01",["A1"]]
         ,["payPrin","acc01",["B"]]
         ,["payIntResidual","acc01","B"]
     ]
     ,"cleanUp":[]
     }
    ,[["CollectedInterest","acc01"]
      ,["CollectedPrincipal","acc01"]
      ,["CollectedPrepayment","acc01"]
      ,["CollectedRecoveries","acc01"]]
    ,None
    ,None
    ,None
    ,None
    ,("PreClosing","Amortizing")
    )

In [165]:
r = localAPI.run(test01
                 ,runAssump = [("stop","2022-04-21")]
                 ,read=True)

Warning Message from server:
Bond A1 is not paid off
Bond B is not paid off

In [161]:
r['bonds']['A1']

,balance,interest,principal,rate,cash,intDue,intOverInt,factor,memo
date,,,,,,,,,
2022-02-20,1200,87.86,0,0.07,87.86,43.23,0,1.0,<PayInt:A1>
2022-03-20,1200,101.53,0,0.07,101.53,0.00,0,1.0,"[<PayInt:A1>, <PayInt:A1>]"
2022-04-20,1200,7.12,0,0.07,7.12,0.00,0,1.0,"[<PayInt:A1>, <PayInt:A1>]"


In [162]:
r['accounts']['acc01'].loc['2022-03-20']

,balance,change,memo
date,,,
2022-03-20,117.86,0.00,<SeqPayFee:trusteeFee>
2022-03-20,69.87,-47.99,<PayInt:A1>
2022-03-20,16.33,-53.54,<PayInt:A1>
2022-03-20,0.00,-16.33,<PayPrin:B>
2022-03-20,0.00,0.00,<PayYield:B>


In [163]:
from absbox import readInspect

readInspect(r['result']).loc["2022-03-20"]

after acc a1 all        101.53
after acc a1 0           47.83
after acc a1 0 ioi        0.16
after acc a1 1           53.01
after acc a1 1 ioi        0.53
after pay 0 a1 all       53.54
after pay 0 a1 0          0.00
after pay 0 a1 0 ioi      0.00
after pay 0 a1 1         53.01
after pay 0 a1 1 ioi      0.53
after pay 1 a1 all        0.00
after pay 1 a1 0          0.00
after pay 1 a1 1          0.00
Name: 2022-03-20, dtype: object

In [71]:
readInspect(r['result'])

,after acc a1 0,after acc a1 1,after acc a1 2
Date,,,
2021-07-26,131.16,131.16,131.16
2021-08-20,136.90,136.90,136.90
2021-09-20,144.02,144.02,144.02
2021-10-20,150.92,150.92,150.92
2021-11-20,158.04,158.04,158.04
2021-12-20,164.94,164.94,164.94
2022-01-20,172.06,172.06,172.06
2022-02-20,179.18,94.40,94.40
2022-03-20,100.84,53.17,0.69
